In [ ]:
#default_exp pretrain

# Pretrain the Barlow Twins model

In [ ]:
#exporti
from fastcore.script import *
from fastai.vision.all import *
from fastai.callback.wandb import *
from self_supervised.augmentations import *
from self_supervised.layers import *
from self_supervised.vision.barlow_twins import *

# fixup depreciation
from kornia import augmentation as korniatfm
korniatfm.GaussianBlur = korniatfm.RandomGaussianBlur
torch.set_num_threads(1)

In [ ]:
#export
@call_parse
def train(
        path:Path, # path
        backbone:str    = 'resnet18', # backbone
        pretrained:bool = False, # pretrained
        size:int        = 224, # size
        projection_size = 2048, # projection_size
        lr:float        = 5e-4, # lr
        epochs:int      = 5, # epochs
        project:str     = None, # project
    ):
    "Pretrain a model on images in `path`."
    import wandb
    wandb.login()

    dls = ImageDataLoaders.from_name_func(path, get_image_files(path), lambda x: False, item_tfms=RandomResizedCrop(size), valid_pct=0)

    fastai_encoder = create_encoder('resnet18', pretrained=pretrained)
    model = create_barlow_twins_model(fastai_encoder, projection_size=projection_size, hidden_size=projection_size)
    aug_pipelines = get_barlow_twins_aug_pipelines(size=size, bw=False, jitter=False)
    if project is None: project = f"{path.name}-barlow-twins-pretraining"
    with wandb.init(project=project, config=dict(
        backbone=backbone, pretrained=pretrained,
        size=size, projection_size=projection_size,
        lr=lr, epochs=epochs)):
        learn = Learner(dls, model, cbs=[BarlowTwins(aug_pipelines), WandbCallback()])
        learn.fit_one_cycle(epochs, lr)
    _pretrained = "-pretrained" if pretrained else ""
    learn.save(f"barlow-twins-{backbone}{_pretrained}-{size}-{epochs}e-proj{projection_size}-lr{lr*1000}e-3",
               with_opt=False)
    return learn

In [ ]:
train(Path('../women-in-tshirts-256/'))

wandb: Currently logged in as: jpcl (use `wandb login --relogin` to force relogin)


Could not gather input dimensions
WandbCallback requires use of "SaveModelCallback" to log best model
WandbCallback was not able to prepare a DataLoader for logging prediction samples -> list index out of range


epoch,train_loss,valid_loss,time


wandb: WARNING GPU stats error Command '['/Users/jpc/.conda/envs/ml-qa/lib/python3.9/site-packages/wandb/bin/apple_gpu_stats', '--json']' died with <Signals.SIGINT: 2>.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇██
mom_0,███████████▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁
raw_loss,▆▆▄▇█▄▅█▄▆▆▆▆▄▇▅▂▆▆▅▆▆▃▃▅▄▅▅▆▃▁▄▂▄▄▁▃▅▁▁
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▇▇▇███▇▇▇▇▇▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁
wd_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,0.4539
eps_0,1e-05
lr_0,0.00016


KeyboardInterrupt: 